## Exercises: Lookups

In the following exercises you will need to use the following terms to make the expressions typecheck:

1. `pure`
2. `(<$>)`
3. `(<*>)`

In [27]:
-- 1.

added :: Maybe Integer
added = (+3) <$> (lookup 3 $ zip [1, 2, 3] [4, 5, 6])

added

-- 2.

y :: Maybe Integer
y = lookup 3 $ zip [1, 2, 3] [4, 5, 6]

z :: Maybe Integer
z = lookup 2 $ zip [1, 2, 3] [4, 5, 6]

tupled :: Maybe (Integer, Integer)
tupled = (,) <$> y <*> z

tupled

-- 3.

import Data.List (elemIndex)

x :: Maybe Int
x = elemIndex 3 [1, 2, 3, 4, 5]

y :: Maybe Int
y = elemIndex 4 [1, 2, 3, 4, 5]

max' :: Int -> Int -> Int
max' = max

maxed :: Maybe Int
maxed = max' <$> x <*> y

maxed

-- 4.

xs = [1, 2, 3]
ys = [4, 5, 6]

x :: Maybe Integer
x = lookup 3 $ zip xs ys

y :: Maybe Integer
y = lookup 2 $ zip xs ys

summed :: Maybe Integer
summed = fmap sum $ (,) <$> x <*> y

Just 9

Just (6,5)

Just 3

## Exercise: Identity Instance

Write an `Applicative` instance for `Identity`.

In [29]:
newtype Identity a =
    Identity a
    deriving (Eq, Ord, Show)

instance Functor Identity where
    fmap f (Identity a) = Identity (f a)

instance Applicative Identity where
    pure = Identity
    (<*>) (Identity f) (Identity a) = Identity (f a)

## Exercise: Constant Instance

Write an `Applicative` instance for `Constant`.

In [31]:
newtype Constant a b =
    Constant { getConstant :: a }
    deriving (Eq, Ord, Show)

instance Functor (Constant a) where
    fmap f (Constant b) = Constant b

instance Monoid a => Applicative (Constant a) where
    pure _ = Constant { getConstant = mempty }
    (<*>) (Constant a) (Constant b) = Constant (mappend a b)